#### Важное требование ко всей домашке в целом: в jupyter ноутбуке не должно был лишнего кода (т.е. если вы взяли за основу семинар, не забудьте удалить все лишнее)

### Requirements

In [3]:
!pip install razdel
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
!pip install FastText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for FastText: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4396452 sha256=857188c9e8f7fb64a4ca8699eafcdffce39b7141caeb8bf1a436bdcf3c5e9944
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built FastText


In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline


from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [134]:
import gensim
from IPython.display import Image
from IPython.core.display import HTML

### Подготовка данных

In [6]:
wiki = open('wiki_data.txt', encoding='utf8').read().split('\n')[:10000]

In [7]:
#Предобработка текста
def preprocess(text):
    tokens = re.sub('#+|[0-9]+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]  #а вот и лемматизация
    return tokens

In [8]:
texts = [preprocess(text) for text in wiki]

In [9]:
vocab = Counter()

for text in texts:
    vocab.update(text)

In [10]:
len(vocab)

142059

In [11]:
vocab.most_common(30)

[('в', 131341),
 ('и', 69108),
 ('', 52982),
 ('год', 46023),
 ('на', 39950),
 ('—', 38394),
 ('с', 31469),
 ('быть', 29100),
 ('он', 18161),
 ('по', 18091),
 ('из', 12693),
 ('который', 11902),
 ('не', 10918),
 ('к', 10302),
 ('а', 9827),
 ('что', 9608),
 ('для', 8846),
 ('от', 8572),
 ('они', 8070),
 ('свой', 8005),
 ('как', 7470),
 ('это', 7115),
 ('за', 6946),
 ('до', 6446),
 ('область', 6328),
 ('первый', 6221),
 ('один', 6131),
 ('после', 6087),
 ('район', 5999),
 ('время', 5980)]

In [12]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

In [13]:
len(filtered_vocab)

7135

In [14]:
list(filtered_vocab)[10:50]

['договориться',
 'никольский',
 'заболеть',
 'культура',
 'выпуск',
 'принадлежащий',
 'подтверждать',
 'уровень',
 'невозможно',
 'девочка',
 'смелый',
 'законопроект',
 'единство',
 'мишель',
 'арестовать',
 'спортсменка',
 'столько',
 'амундсный',
 'цзинчжоу',
 'загрузка',
 'армения',
 'подсемейство',
 'минск',
 'захоронение',
 'берлин',
 'волос',
 'ночной',
 'исполняться',
 'материальный',
 'трибунал',
 'мемориал',
 'механический',
 'ян',
 'атаковать',
 'тихий',
 'свободно',
 'обзор',
 'лисица',
 'просуществовать',
 'лёд']

In [15]:
#Создаем словарь, заменяя слова индексами. Добавляем токен PAD для выравнивания длины предложений
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [17]:
id2word = {i:word for word, i in word2id.items()}

In [18]:
sentences = []

for text in texts:
    tokens = text
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window используется не так и вам нужно это  изменить!)  

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [24]:
vocab_size = len(id2word)
vocab_size

7136

In [22]:
#Посчитаем частотность индексов слов (id_слова + частота)
vocab_ids_count = Counter()
for sent in sentences:
        vocab_ids_count.update(sent)
vocab_ids_count.most_common(5)

[(3001, 131341), (5837, 69108), (1, 52982), (6593, 46023), (2110, 39950)]

In [26]:
#присвоим индексам вероятности, обратные их частотности
probs_vocab_ids = Counter({word:1-(count/sum(vocab_ids_count.values())) for word, count in vocab_ids_count.items()})
probs_vocab_ids[3]

0.999962098758507

In [32]:
# создаем список всех вероятностей для модели
probs_vocab_ids_list = []
for i in range(vocab_size):
    probs_vocab_ids_list.append(probs_vocab_ids[i])
probs_vocab_ids_list[:5]

[0,
 0.9732255523095248,
 0.9999757432054445,
 0.999962098758507,
 0.9999833234537431]

In [28]:
#нормализация вероятностей
p = np.array(probs_vocab_ids_list)
p /= p.sum()

In [35]:
p[:5]

array([0.        , 0.00013642, 0.00014017, 0.00014017, 0.00014017])

## Skip gram

In [38]:
# Функция генерирует батчами обучающие примеры
def gen_batches_sg(sentences, window = 6, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window):i] + sent[i+1:i+window]
                #генерируем положительные примеры
                for context_word in context: 
                    X_target.append(word)
                    X_context.append(context_word) 
                    y.append(1)
                    
                    #и отрицательные
                    X_target.append(word) 
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [39]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model_sg = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_sg.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


model_sg.fit(gen_batches_sg(sentences[:9000], window=6),
          validation_data=gen_batches_sg(sentences[1000:],  window=6),
          batch_size=1000, # количество пар слов в 1 баче обучающей выборки
          steps_per_epoch=5000,
          validation_steps=30,
         epochs=5)

Epoch 1/5
5000/5000 [==============================] - 123s 24ms/step - loss: 0.4709 - accuracy: 0.7813 - val_loss: 0.3918 - val_accuracy: 0.8284
Epoch 2/5
5000/5000 [==============================] - 120s 24ms/step - loss: 0.4294 - accuracy: 0.8096 - val_loss: 0.3978 - val_accuracy: 0.8264
Epoch 3/5
5000/5000 [==============================] - 119s 24ms/step - loss: 0.4496 - accuracy: 0.7994 - val_loss: 0.4524 - val_accuracy: 0.7967
Epoch 4/5
5000/5000 [==============================] - 121s 24ms/step - loss: 0.4386 - accuracy: 0.8039 - val_loss: 0.3985 - val_accuracy: 0.8253
Epoch 5/5
5000/5000 [==============================] - 128s 26ms/step - loss: 0.4206 - accuracy: 0.8131 - val_loss: 0.4137 - val_accuracy: 0.8121


In [40]:
embeddings_sg = model_sg.layers[2].get_weights()[0]

In [41]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [42]:
most_similar('культура', embeddings_sg)

['культура',
 'культурный',
 'архитектурный',
 'архитектура',
 'искусство',
 'археологический',
 'исторический',
 'литература',
 'наследие',
 'памятник']

In [49]:
most_similar('полиция', embeddings_sg)

['полиция',
 'расследование',
 'приговор',
 'преступление',
 'смертный',
 'обязанность',
 'просьба',
 'лично',
 'осудить',
 'заключение']

In [44]:
most_similar('атаковать', embeddings_sg)

['атаковать',
 'боец',
 'утром',
 'фланг',
 'нанести',
 'противник',
 'удар',
 'вражеский',
 'артиллерия',
 'бой']

In [46]:
most_similar('власть', embeddings_sg)

['власть',
 'правительство',
 'государство',
 'подчинение',
 'парламент',
 'подчинённый',
 'независимость',
 'реформа',
 'подчинить',
 'провинциальный']

Получился весьма неплохой результат!

Выводы. Таким образом, обе модели показали хороший результат в генерации эмбеддингов

## CBOW

In [50]:
def gen_batches_cbow(sentences, window = 3, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window):i] + sent[i+1:i+window]

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                X_target.append(np.random.choice(np.array(vocab_size), p = p))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window*2)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [51]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model_cb = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_cb.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [ ]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

# полученную близость нужно преобразовать в вероятность
# когда она одна используется не софтмакс и сигмоида
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model_cb.fit(gen_batches_cbow(sentences[:9000], window=3),
          validation_data=gen_batches_cbow(sentences[1000:],  window=3),
          batch_size=1000,
          steps_per_epoch=4000,
          validation_steps=30,
         epochs=2)

Epoch 1/2
4000/4000 [==============================] - 259s 65ms/step - loss: 0.3133 - accuracy: 0.8664 - val_loss: 0.2002 - val_accuracy: 0.9314
Epoch 2/2
4000/4000 [==============================] - 261s 65ms/step - loss: 0.2346 - accuracy: 0.9033 - val_loss: 0.1789 - val_accuracy: 0.9376


In [54]:
embeddings_cb = model_cb.layers[2].get_weights()[0]

In [56]:
most_similar('культура', embeddings_cb)

['культура',
 'общественный',
 'архитектура',
 'экономический',
 'религия',
 'деятель',
 'хозяйственный',
 'народ',
 'музей',
 'древний']

In [57]:
most_similar('полиция', embeddings_cb)

['полиция',
 'полицейский',
 'милиция',
 'подозревать',
 'сообщить',
 'бандит',
 'боевик',
 'офицер',
 'отражение',
 'чиновник']

In [58]:
most_similar('атаковать', embeddings_cb)

['атаковать',
 'разгромить',
 'вражеский',
 'захватить',
 'повстанец',
 'остановить',
 'прорваться',
 'сдаться',
 'тыл',
 'приказать']

In [59]:
most_similar('власть', embeddings_cb)

['власть',
 'правительство',
 'войско',
 'государство',
 'руководство',
 'союз',
 'управление',
 'закон',
 'автономия',
 'политика']

И снова очень приятный результат!

 Обе модели очень хорошо показали себя в генерации эмбеддингов, и, по ощущениям, нет особых преимуществ ни в скорости, ни в результате. Но если обратить внимание на loss and accuracy, то у CBOW показатели лучше при примерно тех же сумарных показателях временных затрат.

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [85]:
texts[0][:10]

['новостройка',
 'нижегородский',
 'область',
 'новостро́йка',
 '—',
 'сельский',
 'посёлок',
 'в',
 'дивеевский',
 'район']

## Word2Vec

In [83]:
w2v = gensim.models.Word2Vec (texts, 
                              size=300, #vector_size in Gensim
                              min_count=25, # убираем все слова с частотностью меньше 25 раз
                              max_vocab_size=20000,
                              sg=1,  #выбрали Skip-Gram
                              hs=1, #используем hierarchical softmax
                              window=6, 
                              iter=5) #epochs in GenSim

In [79]:
w2v.wv.most_similar("культура")

[('культурный', 0.522752046585083),
 ('памятник', 0.4873464107513428),
 ('доисторический', 0.4821380376815796),
 ('цивилизация', 0.46500295400619507),
 ('наследие', 0.45756006240844727),
 ('народ', 0.4286336898803711),
 ('керамика', 0.4180079698562622),
 ('искусство', 0.40963271260261536),
 ('палеолит', 0.4087953567504883),
 ('археологический', 0.40382450819015503)]

In [80]:
w2v.wv.most_similar('полиция')

[('полицейский', 0.5294318795204163),
 ('расследование', 0.4479517340660095),
 ('подозревать', 0.4453112483024597),
 ('инспектор', 0.43658411502838135),
 ('коррупция', 0.4277951419353485),
 ('жандармерия', 0.4169902205467224),
 ('эрп', 0.4139273166656494),
 ('бандит', 0.40894991159439087),
 ('напарник', 0.4032945930957794),
 ('преступник', 0.3934122323989868)]

In [81]:
w2v.wv.most_similar('атаковать')

[('атака', 0.5351590514183044),
 ('вражеский', 0.4954018294811249),
 ('прорваться', 0.47583216428756714),
 ('соединиться', 0.472461998462677),
 ('ворваться', 0.47025638818740845),
 ('преследовать', 0.46512556076049805),
 ('отступать', 0.4609319865703583),
 ('отрезать', 0.4587652087211609),
 ('противник', 0.4522560238838196),
 ('удар', 0.4449233412742615)]

In [82]:
w2v.wv.most_similar('власть')

[('юрисдикция', 0.43936359882354736),
 ('сянтаня', 0.43816787004470825),
 ('правительство', 0.4289619028568268),
 ('монгол', 0.419599711894989),
 ('чанчж', 0.4177349805831909),
 ('подчинённый', 0.414157509803772),
 ('контроль', 0.40959781408309937),
 ('сяоганя', 0.4072611331939697),
 ('подчинение', 0.4023388624191284),
 ('коммунист', 0.39950770139694214)]

## FastText

In [91]:
ft = gensim.models.FastText(texts, min_n=3, max_n=9)

In [87]:
ft.wv.most_similar('культура')

[('субкультура', 0.9951490163803101),
 ('физкультура', 0.991604208946228),
 ('культуры»', 0.9862306714057922),
 ('культуролог', 0.9853105545043945),
 ('культ', 0.9848628044128418),
 ('росохранкультура', 0.9848345518112183),
 ('историко-культурный', 0.9250214099884033),
 ('культурный', 0.920665979385376),
 ('кулеш', 0.9174954891204834),
 ('археология', 0.9117836952209473)]

In [88]:
ft.wv.most_similar('полиция')

[('алиция', 0.922980785369873),
 ('милиция', 0.9179505109786987),
 ('фелиция', 0.9122324585914612),
 ('галиция', 0.9083214402198792),
 ('полидипсия', 0.8990261554718018),
 ('политрук', 0.880612850189209),
 ('полис', 0.8792989253997803),
 ('поли́тик', 0.871805727481842),
 ('коалиция', 0.8569199442863464),
 ('сульпиция', 0.8370253443717957)]

In [89]:
ft.wv.most_similar('атаковать')

[('штурмовать', 0.9641435146331787),
 ('подорвать', 0.9549635052680969),
 ('сорвать', 0.9543403387069702),
 ('враждовать', 0.9520273208618164),
 ('упаковать', 0.9508740901947021),
 ('оторвать', 0.9497953653335571),
 ('взорвать', 0.949768602848053),
 ('порвать', 0.9489622116088867),
 ('разорвать', 0.9429712295532227),
 ('прорвать', 0.9423761367797852)]

In [90]:
ft.wv.most_similar('власть')

[('«власть', 0.9767284989356995),
 ('власть»', 0.9704247713088989),
 ('класть', 0.7963230609893799),
 ('власов', 0.7925087213516235),
 ('правительство', 0.7813996076583862),
 ('посольство', 0.7653160095214844),
 ('недовольство', 0.7589079141616821),
 ('права»', 0.7503677010536194),
 ('город-государство', 0.7376970052719116),
 ('покровительство', 0.7282108664512634)]

Очень заметно влияние n-gram, поэтому наиболее вероятные эмбеддинги у простых слов иногда получаются крайне странными: полиция - алиция, фелиция, полидипсия. При этом последние два примера получились более-менее (возможно, к ним труднее подобрать рифму?))


# Задание 3 (4 балла)

Используя датасет для классификации (labeled.csv) и простую нейронную сеть (последняя модель в семинаре), оцените качество полученных эмбедингов в задании 1 и 2 (4 набора эмбедингов), также проверьте 1 любую из предобученных моделей с rus-vectores (но только не tayga_upos_skipgram_300_2_2019). 
Какая модель показывает наилучший результат?

Убедитесь, что для каждой модели вы корректно воспроизводите пайплайн предобработки (в 1 задании у вас лемматизация, не забудьте ее применить к датасету для классификации; у выбранной предобученной модели может быть своя специфичная предобработка - ее нужно воспроизвести)

In [93]:
data = pd.read_csv('labeled.csv')

In [94]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\r\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\r\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [95]:
data['norm_text'] = data.comment.apply(preprocess)

In [99]:
data.head()

,comment,toxic,norm_text
0,"Верблюдов-то за что? Дебилы, бл...\r\n",1.0,"[верблюд-то, за, что, дебил, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканый, россиянин, м..."
2,Собаке - собачья смерть\r\n,1.0,"[собака, , собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновить, дебил, это, тоже, не, оск..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[ты, не, убедить, страничный, пдф, в, тот, что..."


In [100]:
# собираем словарь
vocab = Counter()

for text in data['norm_text']:
    vocab.update(text)
    
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)

len(filtered_vocab)

6189

In [101]:
# переводим слова в индексы
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
id2word = {i:word for word, i in word2id.items()}

In [109]:
# создаем список списков индексов из всех текстов датасета
x = []

for tokens in data['norm_text']:
    ids = [word2id[token] for token in tokens if token in word2id]
    x.append(ids)

In [110]:
x[0]

[1247, 3634, 3500]

## Пайплайн для моделей из 1 задания

In [125]:
def tf_embeddings(id2word, emb_model, v_size):
  X = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=v_size)
  y = data.toxic.values

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)
  weights = np.zeros((len(word2id), v_size))

  # получаем веса из последнего слоя модели
  model_weights = emb_model.layers[2].get_weights()[0]

  #вытаскиваем вектора по id 
  for id in id2word.keys(): 
      if id == 0:
          continue
      try:
          weights[id] = model_weights[id]
      
      
      except KeyError:
          continue
          weights[id] = model_weights[12341231234] 

  inputs = tf.keras.layers.Input(shape=(v_size,))


  # передаем матрицу в эмбединг слой
  embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=v_size, 
                                        trainable=False,
                                        weights=[weights])(inputs, )
  mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)

  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  optimizer = tf.keras.optimizers.Adam()
  model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])
  

  # обучаем модель
  model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=20)
  
  return model

In [126]:
cbow = tf_embeddings(id2word, model_cb, 300)

Epoch 1/20
428/428 [==============================] - 3s 5ms/step - loss: 0.6467 - accuracy: 0.6643 - val_loss: 0.6230 - val_accuracy: 0.6810
Epoch 2/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6290 - accuracy: 0.6642 - val_loss: 0.6180 - val_accuracy: 0.6810
Epoch 3/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6278 - accuracy: 0.6642 - val_loss: 0.6171 - val_accuracy: 0.6810
Epoch 4/20
428/428 [==============================] - 3s 6ms/step - loss: 0.6273 - accuracy: 0.6642 - val_loss: 0.6165 - val_accuracy: 0.6810
Epoch 5/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6269 - accuracy: 0.6642 - val_loss: 0.6161 - val_accuracy: 0.6810
Epoch 6/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6265 - accuracy: 0.6643 - val_loss: 0.6155 - val_accuracy: 0.6810
Epoch 7/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6261 - accuracy: 0.6643 - val_loss: 0.6149 - val_accuracy: 0.6810
Epoch 

In [127]:
skipgram = tf_embeddings(id2word, model_sg, 300)

Epoch 1/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6598 - accuracy: 0.6631 - val_loss: 0.6398 - val_accuracy: 0.6630
Epoch 2/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6329 - accuracy: 0.6653 - val_loss: 0.6299 - val_accuracy: 0.6630
Epoch 3/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6283 - accuracy: 0.6653 - val_loss: 0.6279 - val_accuracy: 0.6630
Epoch 4/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6274 - accuracy: 0.6653 - val_loss: 0.6271 - val_accuracy: 0.6630
Epoch 5/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6268 - accuracy: 0.6653 - val_loss: 0.6265 - val_accuracy: 0.6630
Epoch 6/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6263 - accuracy: 0.6653 - val_loss: 0.6260 - val_accuracy: 0.6630
Epoch 7/20
428/428 [==============================] - 2s 4ms/step - loss: 0.6259 - accuracy: 0.6653 - val_loss: 0.6255 - val_accuracy: 0.6630
Epoch 

## Пайплайн для моделей из 2 задания 

In [128]:
#Создадим функцию для моделей gensim
def gensim_emdeddings(word2id, emb_model, v_size):
  X = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=v_size)
  y = data.toxic.values

  # разбиваем датасет на обучающую и тестовую выборки
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)
  # создаем матрицу с векторными представлениями
  weights = np.zeros((len(word2id), v_size))

  for word, i in word2id.items():
      if word == 'PAD':
          continue
      try:
          weights[i] = emb_model.wv[word]
      
      
      except KeyError:        # для слов, которых нет в модели, назначим случайный вектор
          continue
          weights[i] = emb_model.wv['апхчи!']

  inputs = tf.keras.layers.Input(shape=(v_size,))

  # передаем матрицу в эмбединг слой
  embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=v_size, 
                                        trainable=False,
                                        weights=[weights])(inputs, )
  mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)

  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  optimizer = tf.keras.optimizers.Adam()
  model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  # обучаем модель
  model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=20)
  
  return model

In [129]:
FstTxt = gensim_emdeddings(word2id, ft, 100)

Epoch 1/20
428/428 [==============================] - 1s 2ms/step - loss: 0.6278 - accuracy: 0.6537 - val_loss: 0.6056 - val_accuracy: 0.6810
Epoch 2/20
428/428 [==============================] - 1s 3ms/step - loss: 0.6073 - accuracy: 0.6647 - val_loss: 0.5926 - val_accuracy: 0.6824
Epoch 3/20
428/428 [==============================] - 1s 2ms/step - loss: 0.5982 - accuracy: 0.6659 - val_loss: 0.5854 - val_accuracy: 0.6852
Epoch 4/20
428/428 [==============================] - 1s 2ms/step - loss: 0.5916 - accuracy: 0.6691 - val_loss: 0.5797 - val_accuracy: 0.6907
Epoch 5/20
428/428 [==============================] - 1s 2ms/step - loss: 0.5863 - accuracy: 0.6728 - val_loss: 0.5755 - val_accuracy: 0.6963
Epoch 6/20
428/428 [==============================] - 1s 2ms/step - loss: 0.5823 - accuracy: 0.6769 - val_loss: 0.5721 - val_accuracy: 0.6976
Epoch 7/20
428/428 [==============================] - 1s 2ms/step - loss: 0.5786 - accuracy: 0.6792 - val_loss: 0.5700 - val_accuracy: 0.7004
Epoch 

In [130]:
word2vec_classifier = gensim_emdeddings(word2id, w2v, 300)

Epoch 1/20
428/428 [==============================] - 3s 5ms/step - loss: 0.6480 - accuracy: 0.6666 - val_loss: 0.6456 - val_accuracy: 0.6380
Epoch 2/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6222 - accuracy: 0.6666 - val_loss: 0.6423 - val_accuracy: 0.6380
Epoch 3/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6173 - accuracy: 0.6666 - val_loss: 0.6399 - val_accuracy: 0.6380
Epoch 4/20
428/428 [==============================] - 3s 6ms/step - loss: 0.6140 - accuracy: 0.6666 - val_loss: 0.6371 - val_accuracy: 0.6380
Epoch 5/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6109 - accuracy: 0.6666 - val_loss: 0.6345 - val_accuracy: 0.6380
Epoch 6/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6079 - accuracy: 0.6666 - val_loss: 0.6316 - val_accuracy: 0.6380
Epoch 7/20
428/428 [==============================] - 2s 5ms/step - loss: 0.6050 - accuracy: 0.6666 - val_loss: 0.6283 - val_accuracy: 0.6380
Epoch 

## RuWikiRuscorpora_upos_cbow_300_10_2021

In [146]:
Rusvec_model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [147]:
Rusvec_model.most_similar('февраль_NOUN')

[('март_NOUN', 0.9921450614929199),
 ('апрель_NOUN', 0.9915183186531067),
 ('декабрь_NOUN', 0.991229236125946),
 ('октябрь_NOUN', 0.9873937368392944),
 ('ноябрь_NOUN', 0.9869319200515747),
 ('январь_NOUN', 0.984276294708252),
 ('май_NOUN', 0.9827682375907898),
 ('июль_NOUN', 0.9811534881591797),
 ('сентябрь_NOUN', 0.9791117906570435),
 ('июнь_NOUN', 0.9775139689445496)]

In [148]:
mapping = {}


In [149]:
mapping

{}

In [150]:
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [151]:
def normalize_mystem(text):
    tokens = []
    norm_words = m.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue
            
        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma+'_'+pos)

    return tokens

In [152]:
normalize_mystem('Обучить классификатор на продобученной модели дома не получилось')

KeyError: ignored